In [ ]:
# Part1
import cv2
import numpy as np
import os

def detect_shapes_static(image_path, output_dir):
    """Detect and mark shapes in static image with improved color-based detection"""
    
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Read image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Cannot load image {image_path}")
        return
    
    # Convert to HSV for better color detection
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Define precise color ranges for bright/fluorescent colors
    color_ranges = {
        'red': ([0, 100, 100], [10, 255, 255]),      # Red range
        'yellow': ([15, 150, 150], [35, 255, 255]),  # Yellow range
        'green': ([35, 150, 150], [85, 255, 255]),   # Green range  
        'blue': ([100, 150, 150], [130, 255, 255]),  # Blue range
        'magenta': ([140, 150, 150], [180, 255, 255]) # Magenta range
    }
    
    # Process each color
    for color_name, (lower, upper) in color_ranges.items():
        # Create mask for current color
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        
        # Morphological operations to clean up mask
        kernel = np.ones((3, 3), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            area = cv2.contourArea(contour)
            # Increased area threshold to filter out grass noise
            if area < 2000:
                continue
            
            # Check shape compactness to exclude elongated grass shapes
            perimeter = cv2.arcLength(contour, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            if circularity < 0.3:  # Filter out elongated shapes
                continue
            
            # Draw contour outline
            cv2.drawContours(img, [contour], -1, (0, 255, 0), 2)
            
            # Calculate center
            M = cv2.moments(contour)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                
                # Draw center point
                cv2.circle(img, (cx, cy), 5, (255, 0, 0), -1)
                
                # Identify shape based on contour approximation
                epsilon = 0.02 * cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, epsilon, True)
                sides = len(approx)
                
                # Shape classification
                if sides == 3:
                    shape_name = "Triangle"
                elif sides == 4:
                    # Check if it's square or rectangle
                    x, y, w, h = cv2.boundingRect(approx)
                    aspect_ratio = float(w) / h
                    shape_name = "Square" if 0.9 <= aspect_ratio <= 1.1 else "Rectangle"
                elif sides == 5:
                    shape_name = "Pentagon"
                elif sides >= 8:  # More sides = more circular
                    shape_name = "Circle"
                else:
                    shape_name = f"Polygon-{sides}"
                
                # Add label
                cv2.putText(img, f"{color_name} {shape_name}", (cx - 30, cy - 15),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    
    # Save result
    output_path = os.path.join(output_dir, "processed_static.png")
    cv2.imwrite(output_path, img)
    print(f"Static image processing completed. Result saved to: {output_path}")
    
    # Display result briefly and auto-close
    cv2.imshow('Shape Detection Result', img)
    cv2.waitKey(2000)  # Display for 2 seconds then auto-close
    cv2.destroyAllWindows()
    
    return img  # Return the processed image

# Process static image
result_img = detect_shapes_static(
    r"F:\airrobotics\PennAir 2024 App Static.png",
    r"F:\airrobotics"
)

print("Processing completed successfully!")

Static image processing completed. Result saved to: F:\airrobotics\processed_static.png
Processing completed successfully!


In [2]:
# Part 2: Shape Detection on Video (Fixed)

import cv2
import numpy as np
import os

def detect_shapes_video(video_path, output_dir, output_video_name="processed_video.mp4"):
    """Detect and mark shapes in video with real-time processing"""
    
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Open video capture
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video properties: {width}x{height}, {fps} FPS, {total_frames} frames")
    
    # Define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = os.path.join(output_dir, output_video_name)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Define color ranges with improved magenta detection
    color_ranges = {
        'red': ([0, 100, 100], [10, 255, 255]),
        'yellow': ([15, 150, 150], [35, 255, 255]),
        'green': ([35, 150, 150], [85, 255, 255]),
        'blue': ([100, 150, 150], [130, 255, 255]),
        'magenta': ([130, 100, 100], [180, 255, 255])  # Expanded magenta range
    }
    
    frame_count = 0
    shapes_detected_per_frame = []
    
    print("Processing video frames...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % 30 == 0:  # Progress update every 30 frames
            print(f"Processing frame {frame_count}/{total_frames}")
        
        # Create a copy for processing
        processed_frame = frame.copy()
        
        # Convert to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        frame_shapes = []
        
        # Process each color
        for color_name, (lower, upper) in color_ranges.items():
            # Create mask for current color
            mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
            
            # Morphological operations to clean up mask
            kernel = np.ones((3, 3), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
            
            # Find contours
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            for contour in contours:
                area = cv2.contourArea(contour)
                if area < 2000:  # Filter small areas
                    continue
                
                # Check circularity to filter grass
                perimeter = cv2.arcLength(contour, True)
                if perimeter == 0:
                    continue
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                if circularity < 0.3:
                    continue
                
                # Draw contour outline
                cv2.drawContours(processed_frame, [contour], -1, (0, 255, 0), 2)
                
                # Calculate center
                M = cv2.moments(contour)
                if M['m00'] != 0:
                    cx = int(M['m10'] / M['m00'])
                    cy = int(M['m01'] / M['m00'])
                    
                    # Draw center point
                    cv2.circle(processed_frame, (cx, cy), 5, (255, 0, 0), -1)
                    
                    # Identify shape
                    epsilon = 0.02 * cv2.arcLength(contour, True)
                    approx = cv2.approxPolyDP(contour, epsilon, True)
                    sides = len(approx)
                    
                    # Shape classification
                    if sides == 3:
                        shape_name = "Triangle"
                    elif sides == 4:
                        x, y, w, h = cv2.boundingRect(approx)
                        aspect_ratio = float(w) / h
                        shape_name = "Square" if 0.9 <= aspect_ratio <= 1.1 else "Rectangle"
                    elif sides == 5:
                        shape_name = "Pentagon"
                    elif sides >= 8:
                        shape_name = "Circle"
                    else:
                        shape_name = f"Polygon-{sides}"
                    
                    # Add label with black text
                    cv2.putText(processed_frame, f"{color_name} {shape_name}", 
                               (cx - 30, cy - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 2)
                    
                    # Store shape info for tracking
                    frame_shapes.append({
                        'color': color_name,
                        'shape': shape_name,
                        'center': (cx, cy),
                        'area': area
                    })
        
        # Add frame counter with black text
        cv2.putText(processed_frame, f"Frame: {frame_count}/{total_frames}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        # Add shapes count with black text
        cv2.putText(processed_frame, f"Shapes detected: {len(frame_shapes)}", 
                   (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        shapes_detected_per_frame.append(len(frame_shapes))
        
        # Write frame to output video
        out.write(processed_frame)
        
        # Optional: Display frame (comment out for faster processing)
        # cv2.imshow('Video Processing', processed_frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
    
    # Release everything
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    # Print statistics
    avg_shapes = np.mean(shapes_detected_per_frame)
    print(f"\nVideo processing completed!")
    print(f"Output saved to: {output_path}")
    print(f"Average shapes detected per frame: {avg_shapes:.2f}")
    print(f"Total frames processed: {frame_count}")
    
    return output_path

# Process video
video_output = detect_shapes_video(
    r"F:\airrobotics\PennAir 2024 App Dynamic.mp4",
    r"F:\airrobotics",
    "processed_dynamic_video.mp4"
)

print("Video processing completed successfully!")

Video properties: 1920x1080, 30 FPS, 1837 frames
Processing video frames...
Processing frame 30/1837
Processing frame 60/1837
Processing frame 90/1837
Processing frame 120/1837
Processing frame 150/1837
Processing frame 180/1837
Processing frame 210/1837
Processing frame 240/1837
Processing frame 270/1837
Processing frame 300/1837
Processing frame 330/1837
Processing frame 360/1837
Processing frame 390/1837
Processing frame 420/1837
Processing frame 450/1837
Processing frame 480/1837
Processing frame 510/1837
Processing frame 540/1837
Processing frame 570/1837
Processing frame 600/1837
Processing frame 630/1837
Processing frame 660/1837
Processing frame 690/1837
Processing frame 720/1837
Processing frame 750/1837
Processing frame 780/1837
Processing frame 810/1837
Processing frame 840/1837
Processing frame 870/1837
Processing frame 900/1837
Processing frame 930/1837
Processing frame 960/1837
Processing frame 990/1837
Processing frame 1020/1837
Processing frame 1050/1837
Processing fram

In [3]:
# Part 3: Simplified Background Agnostic Shape Detection

import cv2
import numpy as np
import os

def detect_shapes_simplified(video_path, output_dir, output_video_name="processed_simplified.mp4", debug_frames=5):
    """Simplified background agnostic shape detection focusing on key methods"""
    
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Open video capture
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video properties: {width}x{height}, {fps} FPS, {total_frames} frames")
    
    # Define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = os.path.join(output_dir, output_video_name)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_count = 0
    shapes_detected_per_frame = []
    
    print("Processing video frames with simplified detection...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % 30 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
        
        # Create a copy for processing
        processed_frame = frame.copy()
        
        # Apply simplified detection
        detected_shapes = detect_shapes_simplified_frame(frame)
        
        # Draw detected shapes
        for shape in detected_shapes:
            # Draw contour
            cv2.drawContours(processed_frame, [shape['contour']], -1, (0, 255, 0), 2)
            
            # Draw center point
            cv2.circle(processed_frame, shape['center'], 5, (255, 0, 0), -1)
            
            # Add label
            cv2.putText(processed_frame, f"{shape['color']} {shape['shape']}", 
                       (shape['center'][0] - 30, shape['center'][1] - 15), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 2)
        
        # Add frame counter
        cv2.putText(processed_frame, f"Frame: {frame_count}/{total_frames}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        # Add shapes count
        cv2.putText(processed_frame, f"Shapes detected: {len(detected_shapes)}/5", 
                   (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        shapes_detected_per_frame.append(len(detected_shapes))
        
        # Save debug frames
        if frame_count <= debug_frames:
            debug_path = os.path.join(output_dir, f"debug_frame_{frame_count}.png")
            cv2.imwrite(debug_path, processed_frame)
            print(f"Debug frame {frame_count} saved to: {debug_path}")
        
        # Write frame to output video
        out.write(processed_frame)
    
    # Release everything
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    # Print statistics
    avg_shapes = np.mean(shapes_detected_per_frame)
    print(f"\nSimplified processing completed!")
    print(f"Output saved to: {output_path}")
    print(f"Average shapes detected per frame: {avg_shapes:.2f}")
    print(f"Total frames processed: {frame_count}")
    
    return output_path

def detect_shapes_simplified_frame(frame):
    """Simplified multi-method detection focusing on key approaches"""
    
    # Method 1: Enhanced edge detection (most reliable)
    edge_shapes = detect_shapes_enhanced_edge_simple(frame)
    
    # Method 2: Color-based detection with relaxed thresholds
    color_shapes = detect_shapes_color_simple(frame)
    
    # Method 3: Gradient-based detection for smooth shapes
    gradient_shapes = detect_shapes_gradient_simple(frame)
    
    # Combine all methods
    all_shapes = edge_shapes + color_shapes + gradient_shapes
    
    # Simple merging - remove very similar shapes
    merged_shapes = merge_shapes_simple(all_shapes)
    
    return merged_shapes

def detect_shapes_enhanced_edge_simple(frame):
    """Simplified enhanced edge detection"""
    
    shapes = []
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply bilateral filter to reduce noise while preserving edges
    filtered = cv2.bilateralFilter(gray, 9, 75, 75)
    
    # Use multiple edge detection methods
    edges1 = cv2.Canny(filtered, 20, 80)
    edges2 = cv2.Canny(filtered, 40, 120)
    edges3 = cv2.Canny(filtered, 60, 160)
    
    # Combine edges
    edges = cv2.bitwise_or(edges1, edges2)
    edges = cv2.bitwise_or(edges, edges3)
    
    # Morphological operations to close gaps
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 2000 or area > 150000:  # Relaxed size range
            continue
        
        # Check circularity (relaxed)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            continue
        circularity = 4 * np.pi * area / (perimeter * perimeter)
        if circularity < 0.05:  # Very relaxed threshold
            continue
        
        # Check aspect ratio (relaxed)
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if aspect_ratio > 8 or aspect_ratio < 0.125:
            continue
        
        # Identify shape
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        sides = len(approx)
        
        # Detect color
        color_name = detect_color_simple(frame, contour)
        
        # Calculate center
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            
            shape = {
                'contour': contour,
                'center': (cx, cy),
                'area': area,
                'shape': classify_shape_simple(sides, contour),
                'color': color_name,
                'sides': sides,
                'bbox': (x, y, w, h),
                'method': 'edge'
            }
            
            shapes.append(shape)
    
    return shapes

def detect_shapes_color_simple(frame):
    """Simplified color-based detection with relaxed thresholds"""
    
    shapes = []
    
    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define relaxed color ranges for gradient shapes
    color_ranges = {
        'red': ([0, 30, 30], [10, 255, 255]),
        'orange': ([10, 30, 30], [25, 255, 255]),
        'yellow': ([25, 30, 30], [35, 255, 255]),
        'green': ([35, 30, 30], [85, 255, 255]),
        'blue': ([85, 30, 30], [130, 255, 255]),
        'magenta': ([130, 30, 30], [180, 255, 255]),
        'gray': ([0, 0, 30], [180, 30, 200]),
        'white': ([0, 0, 200], [180, 30, 255])
    }
    
    for color_name, (lower, upper) in color_ranges.items():
        # Create mask
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        
        # Morphological operations
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < 2000 or area > 150000:
                continue
            
            # Check circularity (relaxed)
            perimeter = cv2.arcLength(contour, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            if circularity < 0.05:
                continue
            
            # Check aspect ratio (relaxed)
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if aspect_ratio > 8 or aspect_ratio < 0.125:
                continue
            
            # Identify shape
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            sides = len(approx)
            
            # Calculate center
            M = cv2.moments(contour)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                
                shape = {
                    'contour': contour,
                    'center': (cx, cy),
                    'area': area,
                    'shape': classify_shape_simple(sides, contour),
                    'color': color_name,
                    'sides': sides,
                    'bbox': (x, y, w, h),
                    'method': 'color'
                }
                
                shapes.append(shape)
    
    return shapes

def detect_shapes_gradient_simple(frame):
    """Simplified gradient-based detection"""
    
    shapes = []
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate gradients
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    
    # Normalize gradient magnitude
    gradient_magnitude = np.uint8(255 * gradient_magnitude / np.max(gradient_magnitude))
    
    # Apply threshold to get strong gradients
    _, gradient_thresh = cv2.threshold(gradient_magnitude, 30, 255, cv2.THRESH_BINARY)
    
    # Morphological operations
    kernel = np.ones((3, 3), np.uint8)
    gradient_thresh = cv2.morphologyEx(gradient_thresh, cv2.MORPH_CLOSE, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(gradient_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 2000 or area > 150000:
            continue
        
        # Check circularity (relaxed)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            continue
        circularity = 4 * np.pi * area / (perimeter * perimeter)
        if circularity < 0.05:
            continue
        
        # Check aspect ratio (relaxed)
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if aspect_ratio > 8 or aspect_ratio < 0.125:
            continue
        
        # Identify shape
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        sides = len(approx)
        
        # Detect color
        color_name = detect_color_simple(frame, contour)
        
        # Calculate center
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            
            shape = {
                'contour': contour,
                'center': (cx, cy),
                'area': area,
                'shape': classify_shape_simple(sides, contour),
                'color': color_name,
                'sides': sides,
                'bbox': (x, y, w, h),
                'method': 'gradient'
            }
            
            shapes.append(shape)
    
    return shapes

def detect_color_simple(frame, contour):
    """Simplified color detection for gradient shapes"""
    
    # Create mask for the contour area
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [contour], 255)
    
    # Get color statistics from the contour area
    mean_color = cv2.mean(frame, mask=mask)[:3]
    
    # Convert BGR to HSV for better color classification
    bgr_color = np.uint8([[mean_color]])
    hsv_color = cv2.cvtColor(bgr_color, cv2.COLOR_BGR2HSV)[0][0]
    
    h, s, v = hsv_color
    
    # Simplified color classification
    if v > 200 and s < 30:
        return "white"
    elif v > 150 and s < 30:
        return "gray"
    elif h < 10 or h > 170:  # Red range
        return "red"
    elif 10 <= h < 25:  # Orange range
        return "orange"
    elif 25 <= h < 35:  # Yellow range
        return "yellow"
    elif 35 <= h < 85:  # Green range
        return "green"
    elif 85 <= h < 130:  # Blue range
        return "blue"
    elif 130 <= h < 170:  # Magenta/Purple range
        return "magenta"
    else:
        return "unknown"

def classify_shape_simple(sides, contour):
    """Simplified shape classification"""
    
    if sides == 3:
        return "Triangle"
    elif sides == 4:
        # Check aspect ratio for square vs rectangle
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if 0.7 <= aspect_ratio <= 1.3:
            return "Square"
        else:
            return "Rectangle"
    elif sides == 5:
        return "Pentagon"
    elif sides >= 8:
        return "Circle"
    else:
        return f"Polygon-{sides}"

def merge_shapes_simple(all_shapes):
    """Simple shape merging to remove duplicates"""
    
    if len(all_shapes) <= 1:
        return all_shapes
    
    merged_shapes = []
    used_indices = set()
    
    for i, shape1 in enumerate(all_shapes):
        if i in used_indices:
            continue
        
        # Find similar shapes
        similar_shapes = [shape1]
        for j, shape2 in enumerate(all_shapes[i+1:], i+1):
            if j in used_indices:
                continue
            
            # Check if shapes are similar (relaxed criteria)
            if are_shapes_similar_simple(shape1, shape2):
                similar_shapes.append(shape2)
                used_indices.add(j)
        
        # Use the shape with the largest area
        best_shape = max(similar_shapes, key=lambda x: x['area'])
        merged_shapes.append(best_shape)
        used_indices.add(i)
    
    return merged_shapes

def are_shapes_similar_simple(shape1, shape2):
    """Simple shape similarity check"""
    
    # Calculate distance between centers
    cx1, cy1 = shape1['center']
    cx2, cy2 = shape2['center']
    distance = np.sqrt((cx1 - cx2)**2 + (cy1 - cy2)**2)
    
    # Check if shapes are close and have similar properties
    if (distance < 100 and 
        shape1['shape'] == shape2['shape'] and
        abs(shape1['area'] - shape2['area']) < shape1['area'] * 0.8):
        return True
    
    return False

# Process hard video with simplified detection
simplified_output = detect_shapes_simplified(
    r"F:\airrobotics\PennAir 2024 App Dynamic Hard.mp4",
    r"F:\airrobotics",
    "processed_simplified.mp4",
    debug_frames=5
)

print("Simplified processing completed successfully!")


'''
没有解决的问题：
   图像分割；黑色梯形识别不完整
'''

Video properties: 1920x1080, 30 FPS, 1841 frames
Processing video frames with simplified detection...
Debug frame 1 saved to: F:\airrobotics\debug_frame_1.png
Debug frame 2 saved to: F:\airrobotics\debug_frame_2.png
Debug frame 3 saved to: F:\airrobotics\debug_frame_3.png
Debug frame 4 saved to: F:\airrobotics\debug_frame_4.png
Debug frame 5 saved to: F:\airrobotics\debug_frame_5.png
Processing frame 30/1841
Processing frame 60/1841
Processing frame 90/1841
Processing frame 120/1841
Processing frame 150/1841
Processing frame 180/1841
Processing frame 210/1841
Processing frame 240/1841
Processing frame 270/1841
Processing frame 300/1841
Processing frame 330/1841
Processing frame 360/1841
Processing frame 390/1841
Processing frame 420/1841
Processing frame 450/1841
Processing frame 480/1841
Processing frame 510/1841
Processing frame 540/1841
Processing frame 570/1841
Processing frame 600/1841
Processing frame 630/1841
Processing frame 660/1841
Processing frame 690/1841
Processing frame 7

'\n没有解决的问题：\n   图像分割；黑色梯形识别不完整\n'

In [4]:
# Part 4: 3D Shape Detection with Depth Calculation (Complete)

import cv2
import numpy as np
import os

def detect_shapes_3d(video_path, output_dir, output_video_name="processed_3d.mp4", debug_frames=5):
    """3D shape detection with depth calculation using camera intrinsic matrix"""
    
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Open video capture
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {video_path}")
        return
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video properties: {width}x{height}, {fps} FPS, {total_frames} frames")
    
    # Define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = os.path.join(output_dir, output_video_name)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Camera intrinsic matrix (given in the problem)
    K = np.array([
        [2564.3186869, 0, 0],
        [0, 2569.70273111, 0],
        [0, 0, 1]
    ])
    
    # Known radius of the circle (10 inches)
    known_radius = 10.0  # inches
    
    frame_count = 0
    shapes_detected_per_frame = []
    
    print("Processing video frames with 3D detection...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % 30 == 0:
            print(f"Processing frame {frame_count}/{total_frames}")
        
        # Create a copy for processing
        processed_frame = frame.copy()
        
        # Apply 3D detection
        detected_shapes_3d = detect_shapes_3d_frame(frame, K, known_radius)
        
        # Draw detected shapes with 3D coordinates
        for shape in detected_shapes_3d:
            # Draw contour
            cv2.drawContours(processed_frame, [shape['contour']], -1, (0, 255, 0), 2)
            
            # Draw center point
            cv2.circle(processed_frame, shape['center_2d'], 5, (255, 0, 0), -1)
            
            # Add 3D coordinate label
            x_3d, y_3d, z_3d = shape['center_3d']
            cv2.putText(processed_frame, f"{shape['color']} {shape['shape']}", 
                       (shape['center_2d'][0] - 30, shape['center_2d'][1] - 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 2)
            
            # Add 3D coordinates
            cv2.putText(processed_frame, f"3D: ({x_3d:.1f}, {y_3d:.1f}, {z_3d:.1f})", 
                       (shape['center_2d'][0] - 30, shape['center_2d'][1] - 15), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 0, 0), 1)
        
        # Add frame counter
        cv2.putText(processed_frame, f"Frame: {frame_count}/{total_frames}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        # Add shapes count
        cv2.putText(processed_frame, f"Shapes detected: {len(detected_shapes_3d)}/5", 
                   (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        
        shapes_detected_per_frame.append(len(detected_shapes_3d))
        
        # Save debug frames
        if frame_count <= debug_frames:
            debug_path = os.path.join(output_dir, f"debug_3d_frame_{frame_count}.png")
            cv2.imwrite(debug_path, processed_frame)
            print(f"Debug 3D frame {frame_count} saved to: {debug_path}")
        
        # Write frame to output video
        out.write(processed_frame)
    
    # Release everything
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    # Print statistics
    avg_shapes = np.mean(shapes_detected_per_frame)
    print(f"\n3D processing completed!")
    print(f"Output saved to: {output_path}")
    print(f"Average shapes detected per frame: {avg_shapes:.2f}")
    print(f"Total frames processed: {frame_count}")
    
    return output_path

def detect_shapes_3d_frame(frame, K, known_radius):
    """Detect shapes in a frame and calculate 3D coordinates"""
    
    # First detect shapes using 2D methods
    detected_shapes_2d = detect_shapes_simplified_frame(frame)
    
    # Convert to 3D coordinates
    shapes_3d = []
    
    for shape in detected_shapes_2d:
        # Calculate 3D coordinates
        center_3d = calculate_3d_coordinates(shape['center'], K, known_radius, shape['area'])
        
        # Create 3D shape object
        shape_3d = {
            'contour': shape['contour'],
            'center_2d': shape['center'],
            'center_3d': center_3d,
            'area': shape['area'],
            'shape': shape['shape'],
            'color': shape['color'],
            'sides': shape['sides'],
            'bbox': shape['bbox'],
            'method': shape['method']
        }
        
        shapes_3d.append(shape_3d)
    
    return shapes_3d

def calculate_3d_coordinates(center_2d, K, known_radius, area_2d):
    """Calculate 3D coordinates from 2D image coordinates"""
    
    # Extract camera intrinsic parameters
    fx = K[0, 0]  # Focal length in x
    fy = K[1, 1]  # Focal length in y
    cx = K[0, 2]  # Principal point x
    cy = K[1, 2]  # Principal point y
    
    # Get 2D center coordinates
    u, v = center_2d
    
    # Calculate depth (Z) using the known radius
    # For a circle with known radius R, the depth can be calculated as:
    # Z = (f * R) / r, where f is focal length and r is radius in pixels
    
    # Estimate radius in pixels from area
    radius_pixels = np.sqrt(area_2d / np.pi)
    
    # Calculate depth using both focal lengths and average
    depth_x = (fx * known_radius) / radius_pixels
    depth_y = (fy * known_radius) / radius_pixels
    depth = (depth_x + depth_y) / 2  # Average of both calculations
    
    # Calculate 3D coordinates
    # X = (u - cx) * Z / fx
    # Y = (v - cy) * Z / fy
    x_3d = (u - cx) * depth / fx
    y_3d = (v - cy) * depth / fy
    z_3d = depth
    
    return (x_3d, y_3d, z_3d)

def detect_shapes_simplified_frame(frame):
    """Simplified multi-method detection focusing on key approaches"""
    
    # Method 1: Enhanced edge detection (most reliable)
    edge_shapes = detect_shapes_enhanced_edge_simple(frame)
    
    # Method 2: Color-based detection with relaxed thresholds
    color_shapes = detect_shapes_color_simple(frame)
    
    # Method 3: Gradient-based detection for smooth shapes
    gradient_shapes = detect_shapes_gradient_simple(frame)
    
    # Combine all methods
    all_shapes = edge_shapes + color_shapes + gradient_shapes
    
    # Simple merging - remove very similar shapes
    merged_shapes = merge_shapes_simple(all_shapes)
    
    return merged_shapes

def detect_shapes_enhanced_edge_simple(frame):
    """Simplified enhanced edge detection"""
    
    shapes = []
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply bilateral filter to reduce noise while preserving edges
    filtered = cv2.bilateralFilter(gray, 9, 75, 75)
    
    # Use multiple edge detection methods
    edges1 = cv2.Canny(filtered, 20, 80)
    edges2 = cv2.Canny(filtered, 40, 120)
    edges3 = cv2.Canny(filtered, 60, 160)
    
    # Combine edges
    edges = cv2.bitwise_or(edges1, edges2)
    edges = cv2.bitwise_or(edges, edges3)
    
    # Morphological operations to close gaps
    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 2000 or area > 150000:  # Relaxed size range
            continue
        
        # Check circularity (relaxed)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            continue
        circularity = 4 * np.pi * area / (perimeter * perimeter)
        if circularity < 0.05:  # Very relaxed threshold
            continue
        
        # Check aspect ratio (relaxed)
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if aspect_ratio > 8 or aspect_ratio < 0.125:
            continue
        
        # Identify shape
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        sides = len(approx)
        
        # Detect color
        color_name = detect_color_simple(frame, contour)
        
        # Calculate center
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            
            shape = {
                'contour': contour,
                'center': (cx, cy),
                'area': area,
                'shape': classify_shape_simple(sides, contour),
                'color': color_name,
                'sides': sides,
                'bbox': (x, y, w, h),
                'method': 'edge'
            }
            
            shapes.append(shape)
    
    return shapes

def detect_shapes_color_simple(frame):
    """Simplified color-based detection with relaxed thresholds"""
    
    shapes = []
    
    # Convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Define relaxed color ranges for gradient shapes
    color_ranges = {
        'red': ([0, 30, 30], [10, 255, 255]),
        'orange': ([10, 30, 30], [25, 255, 255]),
        'yellow': ([25, 30, 30], [35, 255, 255]),
        'green': ([35, 30, 30], [85, 255, 255]),
        'blue': ([85, 30, 30], [130, 255, 255]),
        'magenta': ([130, 30, 30], [180, 255, 255]),
        'gray': ([0, 0, 30], [180, 30, 200]),
        'white': ([0, 0, 200], [180, 30, 255])
    }
    
    for color_name, (lower, upper) in color_ranges.items():
        # Create mask
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        
        # Morphological operations
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        
        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if area < 2000 or area > 150000:
                continue
            
            # Check circularity (relaxed)
            perimeter = cv2.arcLength(contour, True)
            if perimeter == 0:
                continue
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            if circularity < 0.05:
                continue
            
            # Check aspect ratio (relaxed)
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if aspect_ratio > 8 or aspect_ratio < 0.125:
                continue
            
            # Identify shape
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            sides = len(approx)
            
            # Calculate center
            M = cv2.moments(contour)
            if M['m00'] != 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                
                shape = {
                    'contour': contour,
                    'center': (cx, cy),
                    'area': area,
                    'shape': classify_shape_simple(sides, contour),
                    'color': color_name,
                    'sides': sides,
                    'bbox': (x, y, w, h),
                    'method': 'color'
                }
                
                shapes.append(shape)
    
    return shapes

def detect_shapes_gradient_simple(frame):
    """Simplified gradient-based detection"""
    
    shapes = []
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate gradients
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
    
    # Normalize gradient magnitude
    gradient_magnitude = np.uint8(255 * gradient_magnitude / np.max(gradient_magnitude))
    
    # Apply threshold to get strong gradients
    _, gradient_thresh = cv2.threshold(gradient_magnitude, 30, 255, cv2.THRESH_BINARY)
    
    # Morphological operations
    kernel = np.ones((3, 3), np.uint8)
    gradient_thresh = cv2.morphologyEx(gradient_thresh, cv2.MORPH_CLOSE, kernel)
    
    # Find contours
    contours, _ = cv2.findContours(gradient_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < 2000 or area > 150000:
            continue
        
        # Check circularity (relaxed)
        perimeter = cv2.arcLength(contour, True)
        if perimeter == 0:
            continue
        circularity = 4 * np.pi * area / (perimeter * perimeter)
        if circularity < 0.05:
            continue
        
        # Check aspect ratio (relaxed)
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if aspect_ratio > 8 or aspect_ratio < 0.125:
            continue
        
        # Identify shape
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        sides = len(approx)
        
        # Detect color
        color_name = detect_color_simple(frame, contour)
        
        # Calculate center
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            
            shape = {
                'contour': contour,
                'center': (cx, cy),
                'area': area,
                'shape': classify_shape_simple(sides, contour),
                'color': color_name,
                'sides': sides,
                'bbox': (x, y, w, h),
                'method': 'gradient'
            }
            
            shapes.append(shape)
    
    return shapes

def detect_color_simple(frame, contour):
    """Simplified color detection for gradient shapes"""
    
    # Create mask for the contour area
    mask = np.zeros(frame.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [contour], 255)
    
    # Get color statistics from the contour area
    mean_color = cv2.mean(frame, mask=mask)[:3]
    
    # Convert BGR to HSV for better color classification
    bgr_color = np.uint8([[mean_color]])
    hsv_color = cv2.cvtColor(bgr_color, cv2.COLOR_BGR2HSV)[0][0]
    
    h, s, v = hsv_color
    
    # Simplified color classification
    if v > 200 and s < 30:
        return "white"
    elif v > 150 and s < 30:
        return "gray"
    elif h < 10 or h > 170:  # Red range
        return "red"
    elif 10 <= h < 25:  # Orange range
        return "orange"
    elif 25 <= h < 35:  # Yellow range
        return "yellow"
    elif 35 <= h < 85:  # Green range
        return "green"
    elif 85 <= h < 130:  # Blue range
        return "blue"
    elif 130 <= h < 170:  # Magenta/Purple range
        return "magenta"
    else:
        return "unknown"

def classify_shape_simple(sides, contour):
    """Simplified shape classification"""
    
    if sides == 3:
        return "Triangle"
    elif sides == 4:
        # Check aspect ratio for square vs rectangle
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = float(w) / h
        if 0.7 <= aspect_ratio <= 1.3:
            return "Square"
        else:
            return "Rectangle"
    elif sides == 5:
        return "Pentagon"
    elif sides >= 8:
        return "Circle"
    else:
        return f"Polygon-{sides}"

def merge_shapes_simple(all_shapes):
    """Simple shape merging to remove duplicates"""
    
    if len(all_shapes) <= 1:
        return all_shapes
    
    merged_shapes = []
    used_indices = set()
    
    for i, shape1 in enumerate(all_shapes):
        if i in used_indices:
            continue
        
        # Find similar shapes
        similar_shapes = [shape1]
        for j, shape2 in enumerate(all_shapes[i+1:], i+1):
            if j in used_indices:
                continue
            
            # Check if shapes are similar (relaxed criteria)
            if are_shapes_similar_simple(shape1, shape2):
                similar_shapes.append(shape2)
                used_indices.add(j)
        
        # Use the shape with the largest area
        best_shape = max(similar_shapes, key=lambda x: x['area'])
        merged_shapes.append(best_shape)
        used_indices.add(i)
    
    return merged_shapes

def are_shapes_similar_simple(shape1, shape2):
    """Simple shape similarity check"""
    
    # Calculate distance between centers
    cx1, cy1 = shape1['center']
    cx2, cy2 = shape2['center']
    distance = np.sqrt((cx1 - cx2)**2 + (cy1 - cy2)**2)
    
    # Check if shapes are close and have similar properties
    if (distance < 100 and 
        shape1['shape'] == shape2['shape'] and
        abs(shape1['area'] - shape2['area']) < shape1['area'] * 0.8):
        return True
    
    return False

# Process video with 3D detection
output_3d = detect_shapes_3d(
    r"F:\airrobotics\PennAir 2024 App Dynamic Hard.mp4",
    r"F:\airrobotics",
    "processed_3d.mp4",
    debug_frames=5
)

print("3D processing completed successfully!")

'''
## 3D坐标计算原理

### 相机内参矩阵K的作用：
K = [[fx, 0, cx],
     [0, fy, cy],
     [0, 0, 1]]

其中：
- fx, fy: 焦距（像素单位）
- cx, cy: 主点坐标（图像中心）

### 3D坐标计算公式：
对于图像中的点(u, v)，其3D坐标为：
- X = (u - cx) * Z / fx
- Y = (v - cy) * Z / fy
- Z = 深度（需要计算）

### 深度计算方法：
使用已知的圆形半径（10英寸）来计算深度：
- 从2D面积估算像素半径：r = sqrt(area / π)
- 深度计算：Z = (f * R) / r
- 其中f是焦距，R是已知半径，r是像素半径

### 注意事项：
1. 假设所有形状都在同一平面上
2. 使用圆形作为参考来计算深度
3. 对于非圆形形状，使用面积估算等效半径
4. 坐标单位：英寸（与已知半径单位一致）
'''

Video properties: 1920x1080, 30 FPS, 1841 frames
Processing video frames with 3D detection...
Debug 3D frame 1 saved to: F:\airrobotics\debug_3d_frame_1.png
Debug 3D frame 2 saved to: F:\airrobotics\debug_3d_frame_2.png
Debug 3D frame 3 saved to: F:\airrobotics\debug_3d_frame_3.png
Debug 3D frame 4 saved to: F:\airrobotics\debug_3d_frame_4.png
Debug 3D frame 5 saved to: F:\airrobotics\debug_3d_frame_5.png
Processing frame 30/1841
Processing frame 60/1841
Processing frame 90/1841
Processing frame 120/1841
Processing frame 150/1841
Processing frame 180/1841
Processing frame 210/1841
Processing frame 240/1841
Processing frame 270/1841
Processing frame 300/1841
Processing frame 330/1841
Processing frame 360/1841
Processing frame 390/1841
Processing frame 420/1841
Processing frame 450/1841
Processing frame 480/1841
Processing frame 510/1841
Processing frame 540/1841
Processing frame 570/1841
Processing frame 600/1841
Processing frame 630/1841
Processing frame 660/1841
Processing frame 690/1

'\n## 3D坐标计算原理\n\n### 相机内参矩阵K的作用：\nK = [[fx, 0, cx],\n     [0, fy, cy],\n     [0, 0, 1]]\n\n其中：\n- fx, fy: 焦距（像素单位）\n- cx, cy: 主点坐标（图像中心）\n\n### 3D坐标计算公式：\n对于图像中的点(u, v)，其3D坐标为：\n- X = (u - cx) * Z / fx\n- Y = (v - cy) * Z / fy\n- Z = 深度（需要计算）\n\n### 深度计算方法：\n使用已知的圆形半径（10英寸）来计算深度：\n- 从2D面积估算像素半径：r = sqrt(area / π)\n- 深度计算：Z = (f * R) / r\n- 其中f是焦距，R是已知半径，r是像素半径\n\n### 注意事项：\n1. 假设所有形状都在同一平面上\n2. 使用圆形作为参考来计算深度\n3. 对于非圆形形状，使用面积估算等效半径\n4. 坐标单位：英寸（与已知半径单位一致）\n'